In [15]:
import pandas as pd
import numpy as np
import igraph as ig
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from chart_studio import plotly as py
from plotly.graph_objs import *
from plotly.offline import iplot
from py2neo import Graph

graph = Graph("bolt://localhost:7687", auth=("neo4j", "neo4jneo4j"))

In [16]:
session = graph

In [18]:
edges = list()
edge_w = list()
vertices = dict()

# results = session.run("MATCH (n:Person)-[r:INTERACTS_1|INTERACTS_2|INTERACTS_3|INTERACTS_4]-(m:Person) "
#                       "WHERE(n.community_S1234) IS NOT NULL AND (m.community_S1234) IS NOT NULL "
#                       "RETURN n, m, r")

results = session.run("MATCH (n:Person)-[r:INTERACTS_5|INTERACTS_6]-(m:Person) "
                      "WHERE(n.community_S56) IS NOT NULL AND (m.community_S56) IS NOT NULL "
                      "RETURN n, m, r")

# results = session.run("MATCH (n:Person)-[r:INTERACTS_7|INTERACTS_8]-(m:Person) "\
#                       "WHERE(n.community_S78) IS NOT NULL AND (m.community_S78) IS NOT NULL "\
#                       "RETURN n, m, r")


g = ig.Graph()

for result in results:
  vertices[str(result["n"]["label"])] = str(result["n"]["community_S56"])
  vertices[str(result["m"]["label"])] = str(result["m"]["community_S56"])
  edges.append([str(result["n"]["label"]), str(result["m"]["label"])])
  edge_w.append(1/result["r"]["weight"])

g.add_vertices(list(vertices.keys()))
g.add_edges(edges)
g.es["weight"]=edge_w
g.vs["community"] = list(vertices.values())
N = len(g.vs)
E=[e.tuple for e in g.es]

In [19]:
g.vs[1]

igraph.Vertex(<igraph.Graph object at 0x0000029ABB140850>, 1, {'name': 'Tyrion', 'community': '1'})

In [22]:
jon_id = g.vs.find(name="Jon").index
arya_id = g.vs.find(name="Arya").index
sansa_id = g.vs.find(name="Sansa").index
bran_id = g.vs.find(name="Bran").index

tyrion_id = g.vs.find(name="Tyrion").index
cersei_id = g.vs.find(name="Cersei").index
jaime_id = g.vs.find(name="Jaime").index

daenerys_id = g.vs.find(name="Daenerys").index
stannis_id = g.vs.find(name="Stannis").index

In [23]:
community_list = list(set(list(vertices.values())))
k = len(community_list)

colors = px.colors.qualitative.Plotly[:k]

g.vs['color'] = [None]

color_list= list()

In [24]:
for i in list(vertices.values()):
    color_id = community_list.index(i)
    color_list.append(colors[color_id])

In [25]:
g.vs['color'] = color_list

In [26]:

# Calculate layout
layt = g.layout("fr")

# Define style from network plotting
# visual_style = {}
# visual_style["vertex_size"] = 10
# visual_style["vertex_label_size"] = 5
# visual_style["vertex_color"] = g.vs["color"]
# visual_style["vertex_label"] = g.vs["name"]
# visual_style["layout"] = layt
# visual_style["bbox"] = (1200, 800)
# visual_style["margin"] = 10

# ig.plot(g, **visual_style)


In [28]:
Xn = [layt[k][0] for k in range(N)]
Yn = [layt[k][1] for k in range(N)]
Xe = []
Ye = []
for e in E:
    Xe += [layt[e[0]][0], layt[e[1]][0], None]
    Ye += [layt[e[0]][1], layt[e[1]][1], None]

print(Xn)
print(Yn)

fig = go.Figure()

trace1 = go.Scatter(x=Xe,
                 y=Ye,
                 mode='lines',
                 line=dict(color='rgb(210,210,210)', width=1),
                 hoverinfo='none'
                 )
trace2 = go.Scatter(x=Xn,
                 y=Yn,
                 mode='markers',
                 name= 'People',
                 marker=dict(symbol='circle-dot',
                             size=10,
                             color = g.vs["color"],
                             line=dict(color='rgb(50,50,50)', width=0.5)
                             ),
                 text=g.vs["name"],
                 hoverinfo='text'                 )

fig.add_trace(trace1)
fig.add_trace(trace2)

fig.update_xaxes(showgrid=False, zeroline=False, showticklabels=False)
fig.update_yaxes(showgrid=False, zeroline=False, showticklabels=False)

# axis = dict(showline=False,  # hide axis line, grid, ticklabels and  title
#             zeroline=False,
#             showgrid=False,
#             showticklabels=False,
#             title=''
#             )


width = 800
height = 800

fig.update_layout(title="Game of Thrones Networks for ",
                font=dict(size=12),
                showlegend=False,
                autosize=False,
                width=800,
                height=800,
                margin=layout.Margin(
                    l=40,
                    r=40,
                    b=85,
                    t=100,
                ),
                hovermode='closest',
                # annotations=[
                #     dict(
                #         showarrow=False,
                #         text='This igraph.Graph has the Kamada-Kawai layout',
                #         xref='paper',
                #         yref='paper',
                #         x=0,
                #         y=-0.1,
                #         xanchor='left',
                #         yanchor='bottom',
                #         font=dict(
                #             size=14
                #         )
                #     )
                # ]
                )

fig.add_annotation(
    x=Xn[jon_id],
    y=Yn[jon_id],
    xref="x",
    yref="y",
    text="Jon Snow",
    
)

fig.add_annotation(
    x=Xn[arya_id],
    y=Yn[arya_id],
    xref="x",
    yref="y",
    text="Arya Stark",
)

fig.add_annotation(
    x=Xn[sansa_id],
    y=Yn[sansa_id],
    xref="x",
    yref="y",
    text="Sansa Stark",
)

fig.add_annotation(
    x=Xn[bran_id],
    y=Yn[bran_id],
    xref="x",
    yref="y",
    text="Bran Stark",
)

fig.add_annotation(
    x=Xn[tyrion_id],
    y=Yn[tyrion_id],
    xref="x",
    yref="y",
    text="Tyrion Lannister",
)

fig.add_annotation(
    x=Xn[cersei_id],
    y=Yn[cersei_id],
    xref="x",
    yref="y",
    text="Cersei Lannister",
)

fig.add_annotation(
    x=Xn[jaime_id],
    y=Yn[jaime_id],
    xref="x",
    yref="y",
    text="Jaime Lannister",
)

fig.add_annotation(
    x=Xn[daenerys_id],
    y=Yn[daenerys_id],
    xref="x",
    yref="y",
    text="Daenerys Targaryen",
)

fig.add_annotation(
    x=Xn[stannis_id],
    y=Yn[stannis_id],
    xref="x",
    yref="y",
    text="Stannis Baratheon",
)



# data = [trace1, trace2]
# fig = Figure(data=data, layout=layout)
# py.iplot(fig, filename='Coautorship-network-igraph')
fig.show()

[-3.990168198710767, -1.9081421675263919, -3.410825789153653, -2.210411526515104, -1.359397331879001, -1.5972637568123325, -1.8572186305634417, 0.6431140062326843, -3.095349651844195, -3.679009927397526, -3.5283675857101517, -2.953234274335707, 0.4576386481308503, -3.0205886651895537, -2.584611814942125, -3.092647300225411, -1.9762186296429247, -1.1731520460367326, 1.5430727662259807, 1.2502945926285944, -2.4583438128849284, -1.3632338800857136, -0.5101442018676795, -1.1264310896653538, -1.1308718921544343, -0.8760696606295875, -0.9057000684486808, -1.4502779077855137, 0.8970570517091019, 0.4699566000142705, 1.2734963093742204, 0.8687855882110128, -0.3748890770643997, -2.942023856195226, -3.472133783371364, -1.7119692859117446, -2.515180413072189, -2.816293826983109, 1.2673761274410773, 0.3909744733715639, 2.0446643425218163, 1.756328103755857, 1.1967265303002996, 0.8557689079715292, 2.385092052291002, -2.6390220444761563, -2.7998961610037134, -3.1783180104835074, -3.759014337742918, 3

In [12]:
def plot_community(session, query: str, community: str):
    """Function to plot the network and color the nodes based on the community

    Args:
        query (str): the query to fiter out the nodes and edges to plot
        community (str): the community to color the nodes

    Returns:
        fig: plot of the network
    """


    edges = list()
    edge_w = list()
    vertices = dict()

    results = session.run(query)

    g = ig.Graph()

    for result in results:
        vertices[str(result["n"]["label"])] = str(result["n"][community])
        vertices[str(result["m"]["label"])] = str(result["m"][community])
        edges.append([str(result["n"]["label"]), str(result["m"]["label"])])
        edge_w.append(1/result["r"]["weight"])

    g.add_vertices(list(vertices.keys()))
    g.add_edges(edges)
    g.es["weight"] = edge_w
    g.vs["community"] = list(vertices.values())
    N = len(g.vs)
    E = [e.tuple for e in g.es]
    
    community_list = list(set(list(vertices.values())))


    k = len(community_list)

    colors = px.colors.qualitative.Plotly[:k]

    g.vs['color'] = [None]

    color_list = list()

    for i in list(vertices.values()):
        color_id = community_list.index(i)
        color_list.append(colors[color_id])

    g.vs['color'] = color_list

    layt = g.layout("fr")

    Xn = [layt[k][0] for k in range(N)]


    Yn = [layt[k][1] for k in range(N)]
    Xe = []
    Ye = []

    for e in E:
        Xe += [layt[e[0]][0], layt[e[1]][0], None]
        Ye += [layt[e[0]][1], layt[e[1]][1], None]

    fig = go.Figure()


    trace1 = go.Scatter(x=Xe,
                        y=Ye,
                        mode='lines',
                        line=dict(color='rgb(210,210,210)', width=1),
                        hoverinfo='none'
                        )
    trace2 = go.Scatter(x=Xn,
                        y=Yn,
                        mode='markers',
                        name='ntw',
                        marker=dict(symbol='circle-dot',
                                    size=10,
                                    color=g.vs["color"],
                                    line=dict(color='rgb(50,50,50)', width=0.5)
                                    ),
                        text=g.vs["name"],
                        hoverinfo='text'
                        )

    fig.add_trace(trace1)
    fig.add_trace(trace2)

    fig.update_xaxes(showgrid=False, zeroline=False, showticklabels=False)
    fig.update_yaxes(showgrid=False, zeroline=False, showticklabels=False)

    # axis = dict(showline=False,  # hide axis line, grid, ticklabels and  title
    #             zeroline=False,
    #             showgrid=False,
    #             showticklabels=False,
    #             title=''
    #             )


    fig.update_layout(title="Game of Thrones Networks for ",
                    font=dict(size=12),
                    showlegend=False,
                    autosize=False,
                    width=800,
                    height=800,
                    margin=layout.Margin(
                        l=40,
                        r=40,
                        b=85,
                        t=100,
                    ),
                    hovermode='closest',
                    # annotations=[
                    #     dict(
                    #         showarrow=False,
                    #         text='This igraph.Graph has the Kamada-Kawai layout',
                    #         xref='paper',
                    #         yref='paper',
                    #         x=0,
                    #         y=-0.1,
                    #         xanchor='left',
                    #         yanchor='bottom',
                    #         font=dict(
                    #             size=14
                    #         )
                    #     )
                    # ]
                    )

    # data = [trace1, trace2]
    # fig = Figure(data=data, layout=layout)
    # py.iplot(fig, filename='Coautorship-network-igraph')

    fig.add_annotation(
        x=Xn[jon_id],
        y=Yn[jon_id],
        xref="x",
        yref="y",
        text="Jon Snow",

    )


    fig.add_annotation(
        x=Xn[arya_id],
        y=Yn[arya_id],
        xref="x",
        yref="y",
        text="Arya Stark",
    )

    fig.add_annotation(
        x=Xn[sansa_id],
        y=Yn[sansa_id],
        xref="x",
        yref="y",
        text="Sansa Stark",
    )

    fig.add_annotation(
        x=Xn[bran_id],
        y=Yn[bran_id],
        xref="x",
        yref="y",
        text="Bran Stark",
    )

    fig.add_annotation(
        x=Xn[tyrion_id],
        y=Yn[tyrion_id],
        xref="x",
        yref="y",
        text="Tyrion Lannister",
    )

    fig.add_annotation(
        x=Xn[cersei_id],
        y=Yn[cersei_id],
        xref="x",
        yref="y",
        text="Cersei Lannister",
    )

    fig.add_annotation(
        x=Xn[jaime_id],
        y=Yn[jaime_id],
        xref="x",
        yref="y",
        text="Jaime Lannister",
    )

    fig.add_annotation(
        x=Xn[daenerys_id],
        y=Yn[daenerys_id],
        xref="x",
        yref="y",
        text="Daenerys Targaryen",
    )

    fig.add_annotation(
        x=Xn[stannis_id],
        y=Yn[stannis_id],
        xref="x",
        yref="y",
        text="Stannis Baratheon",
    )

    fig.add_annotation(
        x=Xn[white_walker_id],
        y=Yn[white_walker_id],
        xref="x",
        yref="y",
        text="White Walker",
    )
    
    return fig

In [13]:
query1 = """MATCH (n:Person)-[r:INTERACTS_1|INTERACTS_2|INTERACTS_3|INTERACTS_4]-(m:Person) 
            WHERE(n.community_S1234) IS NOT NULL AND (m.community_S1234) IS NOT NULL 
            RETURN n, m, r"""

query2 = """MATCH (n:Person)-[r:INTERACTS_5|INTERACTS_6]-(m:Person) 
            WHERE(n.community_S56) IS NOT NULL AND (m.community_S56) IS NOT NULL 
            RETURN n, m, r"""

query3 = """MATCH (n:Person)-[r:INTERACTS_7|INTERACTS_8]-(m:Person) 
            WHERE(n.community_S78) IS NOT NULL AND (m.community_S78) IS NOT NULL 
            RETURN n, m, r"""

In [14]:
plot_community(session, query2, "community_S56").show()